# Test API "Flood warnings"

In [1]:
import requests
import json
import pandas as pd
import folium
import geopandas as gpd

In [2]:
url = "http://environment.data.gov.uk/flood-monitoring/id/floods"

### à remplir : latitude, longitude et radius (en km) du point de départ

In [3]:
params = {
    #"lat": 51.507581,
    #"long": -0.127597,
    #"distance": 5,
}

### requête

In [4]:
try:
    api_request = requests.get(url, params=params)
    data = json.loads(api_request.content)
except Exception as e:
    data = "Error..."

### passage à un DataFrame

In [5]:
df = pd.DataFrame(data["items"])

df.head(10)

,@id,description,eaAreaName,eaRegionName,floodArea,floodAreaID,isTidal,message,severity,severityLevel,timeMessageChanged,timeRaised,timeSeverityChanged
0,http://environment.data.gov.uk/flood-monitorin...,River Blythe in Warwickshire,West Midlands,No longer used,{'@id': 'http://environment.data.gov.uk/flood-...,033WAF302,False,,Warning no longer in force,4,2023-10-29T10:41:00,2023-10-29T10:42:00,2023-10-29T10:41:00
1,http://environment.data.gov.uk/flood-monitorin...,River Maun in Nottinghamshire,East Midlands,No longer used,{'@id': 'http://environment.data.gov.uk/flood-...,034WAF430,False,This Flood Alert has been removed. \nRiver l...,Warning no longer in force,4,2023-10-30T08:16:00,2023-10-30T08:16:48,2023-10-30T08:16:00
2,http://environment.data.gov.uk/flood-monitorin...,Middlesbrough Becks,North East,No longer used,{'@id': 'http://environment.data.gov.uk/flood-...,121WAF930,False,Flooding is possible in this area.\n\nMonitor ...,Flood alert,3,2023-10-29T06:43:00,2023-10-29T06:44:01,2023-10-29T06:43:00
3,http://environment.data.gov.uk/flood-monitorin...,River Rye catchment,Yorkshire,No longer used,{'@id': 'http://environment.data.gov.uk/flood-...,122WAF982,False,Flooding is possible in this area.\n\nMonitor ...,Flood alert,3,2023-10-29T10:43:00,2023-10-29T10:43:33,2023-10-29T10:43:00
4,http://environment.data.gov.uk/flood-monitorin...,River Seven catchment,Yorkshire,No longer used,{'@id': 'http://environment.data.gov.uk/flood-...,122WAF980,False,,Warning no longer in force,4,2023-10-30T09:03:00,2023-10-30T09:04:01,2023-10-30T09:03:00
5,http://environment.data.gov.uk/flood-monitorin...,Somerset coast at Porlock Weir,Wessex,No longer used,{'@id': 'http://environment.data.gov.uk/flood-...,112WATSOM1,True,Flooding is possible in this area.\n\nMonitor ...,Flood alert,3,2023-10-28T06:29:00,2023-10-28T06:29:24,2023-10-28T06:29:00
6,http://environment.data.gov.uk/flood-monitorin...,Tidal Thames riverside from Putney Bridge to T...,"Kent, South London and East Sussex",No longer used,{'@id': 'http://environment.data.gov.uk/flood-...,063WAT23West,True,Flooding is possible in this area.\n\nMonitor ...,Flood alert,3,2023-10-25T17:39:00,2023-10-25T17:39:40,2023-10-25T17:39:00
7,http://environment.data.gov.uk/flood-monitorin...,River Till and tributaries,North East,No longer used,{'@id': 'http://environment.data.gov.uk/flood-...,121WAF900,False,Flooding is possible in this area.\n\nMonitor ...,Flood alert,3,2023-10-28T02:42:00,2023-10-28T02:42:58,2023-10-28T02:42:00
8,http://environment.data.gov.uk/flood-monitorin...,River Rede,North East,No longer used,{'@id': 'http://environment.data.gov.uk/flood-...,121WAF907,False,Flooding is possible in this area.\n\nMonitor ...,Flood alert,3,2023-10-27T00:42:00,2023-10-27T00:42:47,2023-10-27T00:42:00
9,http://environment.data.gov.uk/flood-monitorin...,Lower River Coquet,North East,No longer used,{'@id': 'http://environment.data.gov.uk/flood-...,121WAF904,False,Flooding is possible in this area.\n\nMonitor ...,Flood alert,3,2023-10-29T18:43:00,2023-10-29T18:44:01,2023-10-29T18:43:00


### nettoyage des données

In [6]:
# on enlève les colonnes inutiles
columns_to_drop = ['eaRegionName', 'isTidal', 'timeSeverityChanged', 'timeMessageChanged']
df = df.drop(columns=[col for col in columns_to_drop if col in df.columns])

In [7]:
# on ne garde que le polygone
df['floodArea'] = df['floodArea'].apply(lambda x: x['polygon'] if 'polygon' in x else None)

### alertes sur la dernière heure

In [8]:
# affichage des alertes sur la derniere heure
last_hour = df[df['severityLevel'].isin([3, 4])].sort_values(by=['timeRaised']).tail(10)
last_hour

,@id,description,eaAreaName,floodArea,floodAreaID,message,severity,severityLevel,timeRaised
38,http://environment.data.gov.uk/flood-monitorin...,Hunstanton coast in Norfolk,East Anglia,http://environment.data.gov.uk/flood-monitorin...,052WATCSTHUN,This Flood Alert has been removed. \nThere i...,Warning no longer in force,4,2023-10-30T09:07:16
177,http://environment.data.gov.uk/flood-monitorin...,River Churn and its tributaries,Thames,http://environment.data.gov.uk/flood-monitorin...,061WAF02Churn,,Warning no longer in force,4,2023-10-30T09:07:54
166,http://environment.data.gov.uk/flood-monitorin...,River Evenlode from Moreton in Marsh to Cassin...,Thames,http://environment.data.gov.uk/flood-monitorin...,061WAF12Evenlode,,Warning no longer in force,4,2023-10-30T09:11:42
175,http://environment.data.gov.uk/flood-monitorin...,"The River Gipping, from downstream of Needham ...",East Anglia,http://environment.data.gov.uk/flood-monitorin...,054WAFSF4FG,,Warning no longer in force,4,2023-10-30T09:27:08
151,http://environment.data.gov.uk/flood-monitorin...,"The Rivers Blyth and Walpole and Chediston, Br...",East Anglia,http://environment.data.gov.uk/flood-monitorin...,054WAFSF2,,Warning no longer in force,4,2023-10-30T09:27:14
110,http://environment.data.gov.uk/flood-monitorin...,Coast from Gretna to Silloth,Cumbria and Lancashire,http://environment.data.gov.uk/flood-monitorin...,011WACN1,,Warning no longer in force,4,2023-10-30T09:31:03
86,http://environment.data.gov.uk/flood-monitorin...,Coast at Barrow in Furness,Cumbria and Lancashire,http://environment.data.gov.uk/flood-monitorin...,011WACN5,,Warning no longer in force,4,2023-10-30T09:33:56
144,http://environment.data.gov.uk/flood-monitorin...,Upper Bristol Avon area,Wessex,http://environment.data.gov.uk/flood-monitorin...,112WAFTUBA,,Warning no longer in force,4,2023-10-30T09:54:43
99,http://environment.data.gov.uk/flood-monitorin...,"Severn Estuary at Oldbury-on-Severn, Northwick...",Wessex,http://environment.data.gov.uk/flood-monitorin...,112WATSVN2,,Warning no longer in force,4,2023-10-30T09:58:19
243,http://environment.data.gov.uk/flood-monitorin...,Rivers Looe and Seaton,"Devon, Cornwall and the Isles of Scilly",http://environment.data.gov.uk/flood-monitorin...,114WAFT1W06A00,The risk of flooding from the River Pol has n...,Warning no longer in force,4,2023-10-30T10:07:07


### map pour les dernières alertes

In [9]:
# carte avec vue centrée sur le Royaume-Uni
m = folium.Map(location=[53.5, -1.5], zoom_start=6)

for x in last_hour['floodArea']:
    # fichier geojson contenant le polygon
    geojson_file = x
    gdf = gpd.read_file(geojson_file)

    # ajout du polygon sur la carte avec la couleur correspondant au niveau d'alerte (jaune ou rouge) avec bordure de la même couleur
    if last_hour['severityLevel'].iloc[0] == 1:
        folium.GeoJson(gdf, style_function=lambda x: {'fillColor': 'green', 'color': 'green'}).add_to(m)
    elif last_hour['severityLevel'].iloc[0] == 2:
        folium.GeoJson(gdf, style_function=lambda x: {'fillColor': 'yellow', 'color': 'yellow'}).add_to(m)
    elif last_hour['severityLevel'].iloc[0] == 3:
        folium.GeoJson(gdf, style_function=lambda x: {'fillColor': 'orange', 'color': 'orange'}).add_to(m)
    elif last_hour['severityLevel'].iloc[0] == 4:
        folium.GeoJson(gdf, style_function=lambda x: {'fillColor': 'red', 'color': 'red'}).add_to(m)
    else:
        folium.GeoJson(gdf).add_to(m)

# affichage de la carte
display(m)